In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-01"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
142,2024-07-01,Austrália Nbl1 Sul,07:00,Ringwood,Waverley,2.20,1.62,177.5,1.81,1.98,1.5,2.01,1.65,fikK7mfI,0.454545,0.617284,0.552486,0.505051,0.071829,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,171.38,192.40,0.0,0.0,0.0,0.0,0.214724,0.063434,0.139103,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
143,2024-07-01,Austrália Nbl1 Sul,05:00,Ringwood F,Waverley F,2.26,1.59,149.5,1.80,1.88,-2.5,1.95,1.92,lOXYADHD,0.442478,0.628931,0.555556,0.531915,0.071409,0.0,0.0,NaN,NaN,133.912,41.303573,0.308438,NaN,NaN,NaN,0.0,0.0,126.72,0.00,0.0,0.0,0.0,0.0,0.246110,0.030744,0.010963,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
144,2024-07-01,Eua Wnba,23:00,Phoenix Mercury F,Connecticut Sun F,1.98,1.83,168.5,1.88,1.92,-2.5,1.85,2.60,63z2Z2mf,0.505051,0.546448,0.531915,0.520833,0.051499,0.0,0.0,NaN,NaN,NaN,NaN,NaN,123.948,18.388396,0.148356,0.0,0.0,187.05,110.92,0.0,0.0,0.0,1.0,0.055678,0.014886,0.238351,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
145,2024-07-01,Porto Rico Bsn,21:00,Leones De Ponce,Cangrejeros,2.15,1.65,173.5,1.84,1.86,1.5,2.00,1.51,b1mXKyth,0.465116,0.606061,0.543478,0.537634,0.071177,0.0,0.0,NaN,NaN,NaN,NaN,NaN,179.628,29.268049,0.162937,0.0,0.0,207.58,143.56,0.0,0.0,0.0,0.0,0.186081,0.007644,0.197426,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,23:00,Eua Wnba,Phoenix Mercury F,Connecticut Sun F,168.5,1.88,1.0,Over
